<a href="https://colab.research.google.com/github/Daxkansara/Meesho_GST_filing/blob/main/mesov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [237]:
import pandas as pd
import numpy as np

In [238]:
re = pd.read_csv('Reverse.csv')

In [239]:
fp = pd.read_csv('ForwardReports.csv')


In [240]:
# Replace values in column 1 with desired values
re['order_status'] = re['order_status'].replace({'rto': 'negative', 'Cancelled': 'negative', 'Return': 'negative', 'Shipped': 'negative', 'Delivered': 'negative'})

# Set values in column 2 based on values in column 1
re['quantity'] = np.where(re['order_status'] == 'negative', -1 * re['quantity'], re['quantity'])
re['quantity'] = np.where(re['order_status'] == 'positive', 1 * re['quantity'], re['quantity'])

# Set values in column 3 based on values in column 2
re['gst_amount'] = np.where(re['quantity'] < 0, -1 * re['gst_amount'], re['gst_amount'])
re['gst_amount'] = np.where(re['quantity'] > 0, 1 * re['gst_amount'], re['gst_amount'])

# Set values in column 3 based on values in column 2
re['tcs_taxable_amount'] = np.where(re['quantity'] < 0, -1 * re['tcs_taxable_amount'], re['tcs_taxable_amount'])
re['tcs_taxable_amount'] = np.where(re['quantity'] > 0, 1 * re['tcs_taxable_amount'], re['tcs_taxable_amount'])


In [241]:
 #Select the common columns between the dataframes
common_cols = list(set(re.columns).intersection(fp.columns))

# Concatenate the dataframes along the rows with the common columns
df2 = pd.concat([re[common_cols], fp[common_cols]], axis=0, join='inner')

# Save the concatenated dataframe to a new CSV file
#df2.to_csv('combined.csv', index=False)

In [242]:

# Save the modified dataframe to a new CSV file
re.to_csv('modifieddata.csv', index=False)

In [243]:
# Group the data by country and column name, and calculate the sum
df_sum = df2.groupby(['end_customer_state', 'end_customer_state']).sum()

# Pivot the result into a more readable format
df_pivot = df_sum.unstack()
df2.to_csv('gsl1.csv', index=False)

<ipython-input-243-0a1df59d5514>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sum = df2.groupby(['end_customer_state', 'end_customer_state']).sum()


In [244]:
# Loop through all columns and change the keywords to proper case
for col in df2.columns:
    if df2[col].dtype == 'object':
        df2[col] = df2[col].str.title()

In [245]:
# Get the columns that have numeric data
#without gst
numeric_cols = df2.select_dtypes(include='number').columns.tolist()

# Get the unique values in the 'gujarat' column
gujarat_data = df2[df2['end_customer_state_new'].notna()]
gujarat_vals = gujarat_data['end_customer_state_new'].unique()

# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['country'] + numeric_cols)

# Iterate over the unique values in the 'gujarat' column
for val in gujarat_vals:
    # Get the rows that have the current value in the 'gujarat' column
    gujarat_rows = gujarat_data[gujarat_data['end_customer_state_new'] == val]

    # Calculate the sum of numeric columns for the current value
    sum_data = gujarat_rows[numeric_cols].sum(axis=0)

    # Create a new row with the ID and sum data
    new_row = pd.DataFrame([[val] + sum_data.tolist()], columns=['country'] + numeric_cols)

    # Append the new row to the result DataFrame
    result_df = result_df.append(new_row, ignore_index=True)

# Write the result DataFrame to a new CSV file
result_df.to_csv('output_file.csv', index=False)


<ipython-input-245-b5c87af30a37>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-245-b5c87af30a37>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-245-b5c87af30a37>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-245-b5c87af30a37>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-245-b5c87af30a37>:24: FutureWarning: The frame.append method is depre

In [246]:
# Get the columns that have numeric data
numeric_cols = df2.select_dtypes(include='number').columns.tolist()

# Get the unique values in the 'end_customer_state_new' column along with the 'gst_rate' column
gujarat_data = df2[df2['end_customer_state_new'].notna()][['end_customer_state_new', 'gst_rate', 'hsn_code']].drop_duplicates()

# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['country', 'gst_rate', 'hsn_code'] + numeric_cols)

# Iterate over the unique values in the 'end_customer_state_new' column
for val in gujarat_data['end_customer_state_new'].unique():
    # Get the rows that have the current value in the 'end_customer_state_new' column
    gujarat_rows = df2[df2['end_customer_state_new'] == val]

    # Get the unique gst_rate and hsn_code for the current value
    gst_rate = gujarat_data[gujarat_data['end_customer_state_new'] == val]['gst_rate'].iloc[0]
    hsn_code = gujarat_data[gujarat_data['end_customer_state_new'] == val]['hsn_code'].iloc[0]

    # Calculate the sum of numeric columns for the current value
    sum_data = gujarat_rows[numeric_cols].sum(axis=0)

    # Create a new row with the ID, gst_rate, hsn_code, and sum data
    new_row = pd.DataFrame([[val, gst_rate, hsn_code] + sum_data.tolist()], columns=['country', 'gst_rate', 'hsn_code'] + numeric_cols)

    # Append the new row to the result DataFrame
    result_df = result_df.append(new_row, ignore_index=True)

# Write the result DataFrame to a new CSV file
result_df.to_csv('output_file.csv', index=False)


<ipython-input-246-040ddbf18ead>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-246-040ddbf18ead>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-246-040ddbf18ead>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-246-040ddbf18ead>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(new_row, ignore_index=True)
<ipython-input-246-040ddbf18ead>:26: FutureWarning: The frame.append method is depre

In [247]:
mov = pd.read_csv('output_file.csv')

In [248]:
mov.drop(columns=['pin','meesho_price','financial_year','gst_rate.1','taxable_shipping','shipping_gst_18_percent','adj','meesho_price_plus_shipping_charges_total','commission_gst','net_commission','order_num','supplier_id','reseller_pin','gst','shipping_charges_total','end_customer_pin','month_number'],inplace=True)

In [249]:
mov['tcs_taxable_amount'] = mov['tcs_taxable_amount'].round(2)

In [250]:
# Define a function to calculate the IGST, CGST, and SGST values
def calculate_tax(row):
    if row['country'] == 'Gujarat':
        igst = 0
        cgst = row['gst_amount'] / 2
        sgst = row['gst_amount'] / 2
    else:
        igst = row['gst_amount']
        cgst = 0
        sgst = 0
    return pd.Series({'igst': igst, 'cgst': cgst, 'sgst': sgst})

# Apply the function to each row of the DataFrame to calculate the tax values
tax_df = mov.apply(calculate_tax, axis=1)

# Add the new columns to the original DataFrame
df = pd.concat([mov, tax_df], axis=1)

# Write the result DataFrame to a new CSV file
df.to_csv('output_file2.csv', index=False)


In [251]:
sor = pd.read_csv('output_file2.csv')

In [252]:
sor['Total_Transaction_Value'] = sor['tcs_taxable_amount'] + sor['gst_amount']

In [253]:
ko = pd.read_csv('state_conversion.csv')


In [254]:
#ko = pd.read_csv('state_conversion.csv')


state_dict = {
    "35": "Andaman & Nicobar Islands",
    "37": "Andhra Pradesh",
    "12": "Arunachal Pradesh",
    "18": "Assam",
    "10": "Bihar",
    "04": "Chandigarh",
    "22": "Chhattisgarh",
    "26": "Dadra & Nagar Haveli & Daman & Diu",
    "07": "Delhi",
    "30": "Goa",
    "24": "Gujarat",
    "06": "Haryana",
    "02": "Himachal Pradesh",
    "01": "Jammu & Kashmir",
    "20": "Jharkhand",
    "29": "Karnataka",
    "32": "Kerala",
    "31": "Lakshdweep",
    "23": "Madhya Pradesh",
    "27": "Maharashtra",
    "14": "Manipur",
    "17": "Meghalaya",
    "15": "Mizoram",
    "13": "Nagaland",
    "21": "Odisha",
    "34": "Pondicherry",
    "03": "Punjab",
    "08": "Rajasthan",
    "11": "Sikkim",
    "33": "Tamil Nadu",
    "36": "Telengana",
    "16": "Tripura",
    "09": "Uttar Pradesh",
    "05": "Uttarakhand",
    "19": "West Bengal",
    "98": "Other Territory",
    "38": "Ladakh"
}
# Add a new column to the DataFrame with the state names
sor["State Name As Per GSTR-1"] = sor["country"].map(state_dict)


In [255]:


# Perform the outer left join
sor = pd.merge(sor, ko, on='State Name As Per GSTR-1', how='outer')

# Create a dictionary mapping values from 'As Per Meesho' to 'State Name As Per GSTR-1'
state_dict = dict(zip(ko['As Per Meesho'], ko['State Name As Per GSTR-1']))

# Modify the 'State Name As Per GSTR-1' column in the 'sor' DataFrame
sor['State Name As Per GSTR-1'] = sor['country'].str.upper().map(state_dict)

# Group by 'country' and aggregate 'State Name As Per GSTR-1' column as a list
grouped = sor.groupby('country')['State Name As Per GSTR-1'].apply(list)

# Write the grouped DataFrame to a CSV file
grouped.to_csv('output.csv', index=False)


<ipython-input-255-5c1e5d287226>:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  sor = pd.merge(sor, ko, on='State Name As Per GSTR-1', how='outer')


In [256]:
# Perform an outer left join on the 'ID' column
#sor = pd.merge(sor, ko, on='State Name As Per GSTR-1', how='outer')
#state_dict = dict(zip(ko['As Per Meesho'], ko['State Name As Per GSTR-1']))

#sor['State Name As Per GSTR-1'] = sor['country'].str.upper()

#sor['State Name As Per GSTR-1'] = sor['State Name As Per GSTR-1'].map(state_dict)
#sor['State Name As Per GSTR-1'] = ko['State Name As Per GSTR-1'].map(state_dict)
# Write the merged dataframe to a CSV file

#sor.to_csv('output.csv', index=False)



In [257]:
sor = sor.drop(columns=['hsn_code.1', 'As Per Meesho'])

In [258]:
new_column_order = [
    'country',
    'State Name As Per GSTR-1',
    'hsn_code',
    'quantity',
    'tcs_taxable_amount',
    'igst',
    'cgst',
    'sgst',
    'Total_Transaction_Value'
]

# Reorder the columns
sor = sor.reindex(columns=new_column_order)

# Write the DataFrame to a new CSV file
sor.to_csv('my_data_reordered.csv', index=False)